# Module 5 - Programming Assignment

## Directions

There are general instructions on Blackboard and in the Syllabus for Programming Assignments. This Notebook also has instructions specific to this assignment. Read all the instructions carefully and make sure you understand them. Please ask questions on the discussion boards or email me at `EN605.445@gmail.com` if you do not understand something.

<div style="background: mistyrose; color: firebrick; border: 2px solid darkred; padding: 5px; margin: 10px;">
You must follow the directions *exactly* or you will get a 0 on the assignment.
</div>

You must submit a zip file of your assignment and associated files (if there are any) to Blackboard. The zip file will be named after you JHED ID: `<jhed_id>.zip`. It will not include any other information. Inside this zip file should be the following directory structure:

```
<jhed_id>
    |
    +--module-05-programming.ipynb
    +--module-05-programming.html
    +--(any other files)
```

For example, do not name  your directory `programming_assignment_01` and do not name your directory `smith122_pr1` or any else. It must be only your JHED ID.

In [2]:
from IPython.core.display import *
from StringIO import StringIO

## Local Search - Genetic Algorithm

For this assignment we're going to use the Genetic Algorithm to find the solution to a shifted Sphere Function in 10 dimensions, $x$, where the range of $x_i$ in each dimension is (-5.12 to 5.12). Here a "solution" means the vector $x$ that minimizes the function. The Sphere Function is:

$$f(x)=\sum x^2_i$$

We are going to shift it over 0.5 in every dimension:

$$f(x) = \sum (x_i - 0.5)^2$$

where $n = 10$.

As this *is* a minimization problem you'll need to use the trick described in the lecture to turn the shifted Sphere Function into an appropriate fitness function (which is always looking for a *maximum* value).

## Binary GA

You are going to solve the problem two different ways. First, using the traditional (or "Canonical") Genetic Algorithm that encodes numeric values as binary strings (you don't have to represent them literally as strings but they are general lists or strings of only 0 or 1).

There are many different ways to affect this encoding. For this assignment, the easiest is probably to use a 10 bit binary encoding for each $x_i$. This gives each $x_i$ a potential value of 0 to 1024 which can be mapped to (-5.12, 5.12) by subtracting 512 and dividing by 100.

All the GA operators will be as described in the lecture.

**Important**

Please remember that there is a difference between the *genotype* and the *phenotype*. The GA operates on the *genotype* (the encoding) and does not respect the boundaries of the phenotype (the decoding). So, for example, do **not** use a List of Lists to represent an individual. It should be a *single* List of 10 x 10 or 100 bits. In general, crossover and mutation have no idea what those bits represent.

## Real Valued GA

For the real valued GA, you can represent each $x_i$ as a float in the range (-5.12, 5.12) but you will need to create a new mutation operator that applies gaussian noise. Python's random number generator for the normal distribution is called `gauss` and is found in the random module:

```
from random import gauss, random
```

You may need to experiment a bit with the standard deviation of the noise but the mean will be 0.0.

## GA

The Genetic Algorithm itself will have the same basic structure in each case: create a population, evaluate it, select parents, apply crossover and mutation, repeat until the number of desired generations have been generated. The easiest way to accomplish this in "Functional" Python would be to use Higher Order Functions.



Your code should print out the best individual of each generation including the generation number, genotype (the representation), phenotype (the actual value), the fitness (based on your fitness function transformation) and the function value (for the shifted sphere) if passed a DEBUG=True flag.

The GA has a lot of parameters: mutation rate, crossover rate, population size, dimensions (given for this problem), number of generations.  You can put all of those and your fitness function in a `Dict` in which case you need to implement:

```python
def binary_ga( parameters):
  pass
```

and

```python
def real_ga( parameters):
  pass
```

Remember that you need to transform the sphere function into a legit fitness function. Since you also need the sphere function, I would suggest that your parameters Dict includes something like:

```python
parameters = {
   "f": lambda xs: sphere( 0.5, xs),
   "minimization": True
   # put other parameters in here.
}
```

and then have your code check for "minimization" and create an entry for "fitness" that is appropriate.

In [10]:
def sphere( shift, xs):
    return sum( [(x - shift)**2 for x in xs])

In [12]:
sphere( 0.5, [1.0, 2.0, -3.4, 5.0, -1.2, 3.23, 2.87, -4.23, 3.82, -4.61])

113.42720000000001


-----

comment #1

In [8]:
def helper1():
    pass

-----

In [15]:
## Traditional GA
## binary_ga( parameters)

In [16]:
## Real Valued GA
## real_ga( parameters)